In [1]:
%run tools.py

### VGSC analyses - GAARDIAN

In [2]:
df_sample_sets = pd.read_csv("../resources/manifest.tsv", sep="\t")
df_sample_sets

,sample_set,sample_count
0,1191-VO-MULTI-OLOUGHLIN-VMF00106,237
1,1191-VO-MULTI-OLOUGHLIN-VMF00140,1095
2,1244-VO-GH-YAWSON-VMF00149,485


#### Load data

In [3]:
sample_set = "1244-VO-GH-YAWSON-VMF00149"
chroms = ['2L']

In [30]:
snps = {}
pos = {}

for chrom in chroms:
    
    snps[chrom], pos[chrom], metadata = load_arrays_and_metadata("../resources", sample_set, chrom)
    
metadata['location2'] = metadata['location'].str.split(".").str.get(0)

In [5]:
metadata.head(2)

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,species_gambiae_coluzzii,location2
0,VBS45187-5563STDY8782450,WA-2001,Alexander Egyir-Yawson,Ghana,Adumanu.House.A,2018,3,6.304,-1.681,F,gambiae,Adumanu
1,VBS45188-5563STDY8782451,WA-2002,Alexander Egyir-Yawson,Ghana,Adumanu.House.C,2018,10,6.302,-1.682,F,coluzzii,Adumanu


#### Calculate VGSC allele freqs

In [22]:
mutations = pd.read_csv("../../IR/rna-seq-busia/resources/exampleMutations.tsv", sep="\t")
vgsc = mutations.query("Gene == 'VGSC'")
vgsc['Position'] = vgsc['Location'].str.split("-").str.get(1)
vgsc

/home/sanj/apps/anaconda3/envs/pysanj/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/sanj/apps/anaconda3/envs/pysanj/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Gene,GeneID,ALT,ALT2,Name,Location,Position
0,VGSC,AGAP004707,C,T,V402L,2L:2391228-2391228,2391228
1,VGSC,AGAP004707,T,NaN,T791M,2L:2416980-2416980,2416980
2,VGSC,AGAP004707,C,NaN,L995S,2L:2422651-2422651,2422651
3,VGSC,AGAP004707,T,NaN,L995F,2L:2422652-2422652,2422652
4,VGSC,AGAP004707,C,NaN,I1527T,2L:2429617-2429617,2429617
5,VGSC,AGAP004707,T,NaN,N1570Y,2L:2429745-2429745,2429745
6,VGSC,AGAP004707,T,NaN,P1874S,2L:2430880-2430880,2430880
7,VGSC,AGAP004707,T,NaN,P1874L,2L:2430881-2430881,2430881


In [11]:
coluzzii_bool = metadata.species_gambiae_coluzzii == 'coluzzii'
geno = snps[chrom].compress(coluzzii_bool, axis=1)
meta = metadata[coluzzii_bool]

In [45]:
for idx, row in vgsc.iterrows():
    
    mut = row['Name']
    position = row['Position']
    
    bool_ = pos[chrom] == int(position)
    
    gn = geno.compress(bool_, axis=0)
    freq = gn.count_alleles().to_frequencies()
    
    print(mut, freq.compute())

V402L [[0.86492891 0.         0.07938389 0.0556872 ]]
T791M [[0.98933649 0.         0.01066351]]
L995S [[1.]]
L995F [[0.13507109 0.         0.86492891]]
I1527T [[0.86492891 0.         0.13507109]]
N1570Y [[1.]]
P1874S [[0.66469194 0.         0.33530806]]
P1874L [[0.99052133 0.         0.00947867]]


In [46]:
gamb_bool = metadata.species_gambiae_coluzzii == 'gambiae'
geno = snps[chrom].compress(gamb_bool, axis=1)
meta = metadata[gamb_bool]

In [48]:
for idx, row in vgsc.iterrows():
    
    mut = row['Name']
    position = row['Position']
    
    bool_ = pos[chrom] == int(position)
    
    gn = geno.compress(bool_, axis=0)
    freq = gn.count_alleles().to_frequencies()
    
    print(mut, freq.compute())

V402L [[1.]]
T791M [[0.63492063 0.         0.36507937]]
L995S [[1.]]
L995F [[0. 0. 1.]]
I1527T [[1.]]
N1570Y [[0.88888889 0.         0.11111111]]
P1874S [[0.99206349 0.00793651]]
P1874L [[0.72222222 0.         0.27777778]]


In [50]:
?allel.write_vcf

In [54]:
geno= geno[:1000]

In [57]:
pos

{'2L': array([    1215,     1216,     1231, ..., 49356403, 49356404, 49359100],
       dtype=int32)}

In [56]:
pos[:1000]

TypeError: unhashable type: 'slice'

In [58]:
callset = {'CHROM': '2L', 'POS': pos['2L'][:1000], "calldata/GT": geno}



In [59]:
allel.write_vcf("test.vcf", callset)

In [60]:
ls

Fst_fm_coluzzii.tsv             GAARDIAN_VGSC.ipynb       test.vcf
G12_GAARDIAN_all_samples.ipynb  Karyotype_gaardian.ipynb  tools.py
GAARDIAN_Fst.ipynb              old/
GAARDIAN_PCA.ipynb              PCA_GAARDIAN.ipynb


In [61]:
cat test.vcf

##fileformat=VCFv4.1
##fileDate=20211108
##source=scikit-allel-1.2.1
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO
2	1215	.	.	.	.	.	.
L	1216	.	.	.	.	.	.
